In [6]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from datetime import datetime

API_KEY = '2acb79dd-f8ca-48b5-b10c-bbee2290ed94'
BASE_URL = 'https://api.tokenterminal.com/v2'

def fetch_projects():
    url = f'{BASE_URL}/projects'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def fetch_metrics():
    url = f'{BASE_URL}/metrics'
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def fetch_time_series_data(metric_ids, project_ids, start_date, end_date):
    cache = {project_id: [] for project_id in project_ids}
    project_ids_str = ','.join(project_ids)
    
    for metric_id in metric_ids:
        url = f'{BASE_URL}/metrics/{metric_id}'
        headers = {
            'Authorization': f'Bearer {API_KEY}'
        }
        params = {
            'project_ids': project_ids_str,
            'start': start_date,
            'end': end_date
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 404:
            print(f"Data for projects or metric ID '{metric_id}' not found.")
            continue
        response.raise_for_status()
        data = response.json()['data']
        
        for entry in data:
            cache[entry['project_id']].append(entry)
    
    for project_id in cache:
        if cache[project_id]:
            df = pd.DataFrame(cache[project_id])
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df = df.pivot(index='timestamp', columns='metric_id', values='value')
            df.reset_index(inplace=True)
            if 'market_cap_circulating' in df.columns and 'market_cap_fully_diluted' in df.columns:
                df['circulating_supply'] = df['market_cap_circulating'] / df['market_cap_fully_diluted']
            cache[project_id] = df
        else:
            cache[project_id] = pd.DataFrame()
    
    return cache

def add_total_and_rolling_averages(data_dict):
    total_df = pd.DataFrame()
    for project_id, df in data_dict.items():
        if not df.empty:
            df['rolling_30_days_circulating'] = df['market_cap_circulating'].rolling(window=30, min_periods=1).mean()
            df['rolling_30_days_diluted'] = df['market_cap_fully_diluted'].rolling(window=30, min_periods=1).mean()
            df['rolling_180_days_circulating'] = df['market_cap_circulating'].rolling(window=180, min_periods=1).mean()
            df['rolling_180_days_diluted'] = df['market_cap_fully_diluted'].rolling(window=180, min_periods=1).mean()
            df['30_day_percent_change_circulating'] = df['market_cap_circulating'] / df['rolling_30_days_circulating']
            df['30_day_percent_change_diluted'] = df['market_cap_fully_diluted'] / df['rolling_30_days_diluted']
            df['180_day_percent_change_circulating'] = df['market_cap_circulating'] / df['rolling_180_days_circulating']
            df['180_day_percent_change_diluted'] = df['market_cap_fully_diluted'] / df['rolling_180_days_diluted']
            
            if total_df.empty:
                total_df = df[['timestamp']].copy()
                total_df['market_cap_circulating'] = 0
                total_df['market_cap_fully_diluted'] = 0
            
            total_df = pd.merge(total_df, df[['timestamp', 'market_cap_circulating', 'market_cap_fully_diluted']], on='timestamp', how='outer', suffixes=('', f'_{project_id}'))
            total_df['market_cap_circulating'] += df['market_cap_circulating'].fillna(0)
            total_df['market_cap_fully_diluted'] += df['market_cap_fully_diluted'].fillna(0)
    
    total_df['circulating_supply'] = total_df['market_cap_circulating'] / total_df['market_cap_fully_diluted']
    total_df['rolling_30_days_circulating'] = total_df['market_cap_circulating'].rolling(window=30, min_periods=1).mean()
    total_df['rolling_30_days_diluted'] = total_df['market_cap_fully_diluted'].rolling(window=30, min_periods=1).mean()
    total_df['rolling_180_days_circulating'] = total_df['market_cap_circulating'].rolling(window=180, min_periods=1).mean()
    total_df['rolling_180_days_diluted'] = total_df['market_cap_fully_diluted'].rolling(window=180, min_periods=1).mean()
    total_df['30_day_percent_change_circulating'] = total_df['market_cap_circulating'] / total_df['rolling_30_days_circulating']
    total_df['30_day_percent_change_diluted'] = total_df['market_cap_fully_diluted'] / total_df['rolling_30_days_diluted']
    total_df['180_day_percent_change_circulating'] = total_df['market_cap_circulating'] / total_df['rolling_180_days_circulating']
    total_df['180_day_percent_change_diluted'] = total_df['market_cap_fully_diluted'] / total_df['rolling_180_days_diluted']
    total_df = total_df[['timestamp', 'market_cap_circulating', 'market_cap_fully_diluted', 'circulating_supply', 
                         'rolling_30_days_circulating', 'rolling_30_days_diluted', 
                         '30_day_percent_change_circulating', '30_day_percent_change_diluted', 
                         'rolling_180_days_circulating', 'rolling_180_days_diluted', 
                         '180_day_percent_change_circulating', '180_day_percent_change_diluted']].copy()
    total_df.sort_values('timestamp', inplace=True)
    total_df.reset_index(drop=True, inplace=True)
    data_dict['total'] = total_df

    for project_id, df in data_dict.items():
        if project_id != 'total' and not df.empty:
            df = pd.merge(df, total_df[['timestamp', '30_day_percent_change_circulating', '180_day_percent_change_circulating']], on='timestamp', suffixes=('', '_market'))
            df['30_day_change_vs_market'] = df['30_day_percent_change_circulating'] / df['30_day_percent_change_circulating_market']
            df['180_day_change_vs_market'] = df['180_day_percent_change_circulating'] / df['180_day_percent_change_circulating_market']
            df.drop(columns=['30_day_percent_change_circulating_market', '180_day_percent_change_circulating_market'], inplace=True)
            data_dict[project_id] = df
    
    return data_dict

def plot_circulating_supply_and_changes(data_dict):
    fig = go.Figure()

    for project_id, data in data_dict.items():
        if 'circulating_supply' in data.columns and '30_day_change_vs_market' in data.columns:
            data.set_index('timestamp', inplace=True)
            
            fig.add_trace(go.Scatter(
                x=data.index, y=data['circulating_supply'], mode='lines+markers', name=f'{project_id} Circulating Supply',
                yaxis='y1'
            ))
            
            fig.add_trace(go.Scatter(
                x=data.index, y=data['30_day_change_vs_market'], mode='lines', name=f'{project_id} 30-Day Change vs Market',
                yaxis='y2'
            ))
            
            fig.add_trace(go.Scatter(
                x=data.index, y=data['30_day_percent_change_circulating'], mode='lines', name=f'{project_id} 30-Day Change',
                yaxis='y3'
            ))

    fig.update_layout(
        title='Circulating Supply, 30-Day Change vs Basket, and 30 day supply change for Selected Projects',
        xaxis_title='Date',
        yaxis=dict(
            title='Circulating Supply',
            side='left'
        ),
        yaxis2=dict(
            title='30-Day Change vs Market',
            side='right',
            overlaying='y',
            anchor='free',
            position=0.85
        ),
        yaxis3=dict(
            title='30-Day Supply Change',
            side='right',
            overlaying='y',
            anchor='free',
            position=0.95
        ),
        legend_title='Metric',
        template='plotly_white'
    )
    fig.show()

projects = fetch_projects()
metrics = fetch_metrics()

projects_df = pd.DataFrame(projects['data'])
metrics_df = pd.DataFrame(metrics['data'])

metric_ids = ['price','market_cap_circulating','market_cap_fully_diluted']

project_ids = ['dydx','gmx','kwenta','apex','apollox','gains-network','vertex-protocol','pendle','holdstation','perpetual-protocol','lyra']
start_date = '2022-01-01'
end_date = '2024-05-28'

time_series_data = fetch_time_series_data(metric_ids, project_ids, start_date, end_date)
time_series_data = add_total_and_rolling_averages(time_series_data)
plot_circulating_supply_and_changes(time_series_data)
